In [1]:
# 파이썬 버전 체크
import sys
sys.version 

'3.10.14 | packaged by Anaconda, Inc. | (main, May  6 2024, 19:44:50) [MSC v.1916 64 bit (AMD64)]'

In [2]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

In [13]:
from langchain_teddynote import logging

# LangSmith 추적 시작
#logging.langsmith("problem_content_agent_TEST")

# LangSmith 추적 끝
#logging.langsmith("problem_content_agent_TEST", set_enable=False)

LangSmith 추적을 하지 않습니다.


In [4]:
import pandas as pd

# 데이터 프레임 불러오기
df = pd.read_csv("dataset/Leetcode20.csv")

In [5]:
df.head(2)

,problem_number,title,difficulty,content
0,1,Two Sum,Easy,Given an array of integers `nums` and an integ...
1,2,Add Two Numbers,Medium,You are given two **non-empty** linked lists r...


In [7]:
import os
from langchain.agents import Tool
from langchain_community.chat_models.openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults

# LLM 모델 = gpt-4o
gpt4o = ChatOpenAI(model_name="gpt-4o", api_key=os.getenv("OPENAI_API_KEY"))

# TavilySearchResults 클래스의 인스턴스를 생성합니다 (검색 결과 최대 3개)
search = TavilySearchResults(api_key=os.environ['TAVILY_API_KEY'], max_results=3)

# Tavily Search tool 생성
tavily_search = Tool(
    name="tavily_search",
    func=lambda q: search.run(q),
    description="Use this tool to search for Leetcode problems numbered above 20 or problems from other coding test sites."
)

In [8]:
from langchain_experimental.agents import create_pandas_dataframe_agent
from langchain.agents.agent_types import AgentType

# Leetcode 1-20번 문제 : 데이터 프레임 검색
# Leetcode 21번 이상 문제 or 타 코딩사이트 문제 : 인터넷 검색
custom_prefix = """
Provide detailed and easy-to-understand explanations of coding test problems.

Constraints:
- If the problem is a Leetcode problem between 1 and 20, search for it in the CSV file named 'df'. Ensure the correct problem number is referenced based on user input, avoiding any confusion.
- For Leetcode problems above 20 or problems from other coding platforms, use Tavily to perform the search. When using Tavily, do not return just a link; instead, read and summarize the content in a detailed and user-friendly manner.

Expected Output:
- A clear description of the problem's objective.
- Explanation of the input and output formats.
- Key constraints and edge cases to consider.
- Possible strategies or algorithms to solve the problem (without providing full code).
- If searching via Tavily, include a thorough explanation that covers the problem's details based on the source material, avoiding any reliance on external links alone.
"""

# pandas dataframe agent 생성
problem_content_agent = create_pandas_dataframe_agent(
    llm=gpt4o,
    df=df,
    agent_type=AgentType.OPENAI_FUNCTIONS,
    allow_dangerous_code=True,
    verbose=True,
    extra_tools=[tavily_search],
    prefix = custom_prefix
)

In [9]:
from IPython.display import display, Markdown

# 데이터 프레임 Leetcode 12번 문제 내용
display(Markdown(df["content"][11]))

Roman numerals are represented by seven different symbols: `I`, `V`, `X`, `L`, `C`, `D` and `M`.

**Symbol**       **Value**
I             1
V             5
X             10
L             50
C             100
D             500
M             1000

For example, `2` is written as `II` in Roman numeral, just two one's added together. `12` is written as `XII`, which is simply `X + II`. The number `27` is written as `XXVII`, which is `XX + V + II`.

Roman numerals are usually written largest to smallest from left to right. However, the numeral for four is not `IIII`. Instead, the number four is written as `IV`. Because the one is before the five we subtract it making four. The same principle applies to the number nine, which is written as `IX`. There are six instances where subtraction is used:

*   `I` can be placed before `V` (5) and `X` (10) to make 4 and 9.
*   `X` can be placed before `L` (50) and `C` (100) to make 40 and 90.
*   `C` can be placed before `D` (500) and `M` (1000) to make 400 and 900.

Given an integer, convert it to a roman numeral.

**Example 1:**

**Input:** num = 3
**Output:**  "III "
**Explanation:** 3 is represented as 3 ones.

**Example 2:**

**Input:** num = 58
**Output:**  "LVIII "
**Explanation:** L = 50, V = 5, III = 3.

**Example 3:**

**Input:** num = 1994
**Output:**  "MCMXCIV "
**Explanation:** M = 1000, CM = 900, XC = 90 and IV = 4.

**Constraints:**

*   `1 <= num <= 3999`

In [11]:
# agent invoke 결과 : Leetcode 12번 문제 설명 해줘 
result = problem_content_agent.invoke({"input": "Please explain the content of Leetcode problem 12."})
display(Markdown(result['output']))



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "df[df['problem_number'] == 12]"}`


    problem_number             title difficulty  \
11              12  Integer to Roman     Medium   

                                              content  
11  Roman numerals are represented by seven differ...  ### Leetcode Problem 12: Integer to Roman

**Objective:**
Convert a given integer to its corresponding Roman numeral representation.

**Roman Numerals:**
Roman numerals are represented by seven different symbols:
- `I` (1)
- `V` (5)
- `X` (10)
- `L` (50)
- `C` (100)
- `D` (500)
- `M` (1000)

Roman numerals are usually written from largest to smallest from left to right. However, there are exceptions where smaller numbers precede larger numbers to indicate subtraction:
- `IV` (4) instead of `IIII`
- `IX` (9) instead of `VIIII`
- `XL` (40) instead of `XXXX`
- `XC` (90) instead of `LXXXX`
- `CD` (400) instead of `CCCC`
- `CM` (900) instead of `DCCCC`

**Input

### Leetcode Problem 12: Integer to Roman

**Objective:**
Convert a given integer to its corresponding Roman numeral representation.

**Roman Numerals:**
Roman numerals are represented by seven different symbols:
- `I` (1)
- `V` (5)
- `X` (10)
- `L` (50)
- `C` (100)
- `D` (500)
- `M` (1000)

Roman numerals are usually written from largest to smallest from left to right. However, there are exceptions where smaller numbers precede larger numbers to indicate subtraction:
- `IV` (4) instead of `IIII`
- `IX` (9) instead of `VIIII`
- `XL` (40) instead of `XXXX`
- `XC` (90) instead of `LXXXX`
- `CD` (400) instead of `CCCC`
- `CM` (900) instead of `DCCCC`

**Input:**
- An integer `num` (1 <= `num` <= 3999).

**Output:**
- A string representing the Roman numeral of the given integer.

**Key Constraints:**
- The input integer will always be between 1 and 3999, inclusive.

**Example 1:**
- **Input:** `num = 3`
- **Output:** `"III"`
- **Explanation:** 3 is represented by three `I`s.

**Example 2:**
- **Input:** `num = 58`
- **Output:** `"LVIII"`
- **Explanation:** 50 is `L`, 5 is `V`, and 3 is represented by three `I`s.

**Example 3:**
- **Input:** `num = 1994`
- **Output:** `"MCMXCIV"`
- **Explanation:** 1000 is `M`, 900 is `CM`, 90 is `XC`, and 4 is `IV`.

**Possible Strategies:**
1. **Greedy Approach:**
   - Create a list of tuples containing the Roman numeral symbols and their corresponding values in descending order.
   - Iterate through this list, subtracting the largest possible values from `num` and appending the corresponding symbols to the result string until `num` is reduced to zero.

2. **Mapping Approach:**
   - Use a dictionary to map integers to their corresponding Roman numeral symbols.
   - Iterate through the integer from the highest place value down to the lowest, constructing the Roman numeral by checking against special cases and concatenating symbols.

**Edge Cases:**
- Ensure that special subtraction cases (like 4, 9, 40, 90, etc.) are handled correctly.
- Test for the smallest (`num = 1`) and largest (`num = 3999`) possible inputs to confirm the solution works across the full range.

By adhering to these strategies and constraints, you can effectively convert integers to Roman numerals.

In [12]:
# agent invoke 결과 : Leetcode 200번 문제 설명 해줘 
result = problem_content_agent.invoke({"input": "Please explain the content of Leetcode problem 200."})
display(Markdown(result['output']))



> Entering new AgentExecutor chain...

Invoking: `tavily_search` with `Leetcode problem 200`


[{'url': 'https://livefiredev.com/number-of-islands-leetcode-explained-with-animations-visualizations/', 'content': "In this guide, we will look at the Leet Code problem: 200.Number of Islands. As you probably know this is an important problem since it also features in the Blind Curated List of Top 75 LeetCode Questions to Save Your Time. This was a list created by a Facebook engineer to help people prepare for Coding Interviews. It's a very popular list of just 75 questions from Leetcode."}, {'url': 'https://algo.monster/liteproblems/200', 'content': 'In-depth solution and explanation for LeetCode 200. Number of Islands in Python, Java, C++ and more. Intuitions, example walk through, and complexity analysis. Better than official and forum solutions.'}, {'url': 'https://leetcode.com/problems/number-of-islands/', 'content': "Test Result. 200. Number of Islands. Medium. Given an m x n 2D bina

### Leetcode Problem 200: Number of Islands

#### Problem Description
The problem is to determine the number of islands in a given 2D binary grid. An island is defined as a group of `1`s (representing land) that are connected horizontally or vertically. The grid is surrounded by water represented by `0`s.

#### Objective
Given an `m x n` 2D binary grid `grid`, return the number of islands. An island is formed by connecting adjacent lands horizontally or vertically, and it is surrounded by water.

#### Input Format
- `grid`: A 2D list of integers where:
  - `grid[i][j]` is either `1` (land) or `0` (water).

#### Output Format
- An integer representing the number of islands.

#### Constraints
- The grid dimensions are `m x n` where `1 <= m, n <= 300`.
- The grid contains only `0`s and `1`s.

#### Example
```python
Input: grid = [
  ["1","1","1","1","0"],
  ["1","1","0","1","0"],
  ["1","1","0","0","0"],
  ["0","0","0","0","0"]
]
Output: 1

Input: grid = [
  ["1","1","0","0","0"],
  ["1","1","0","0","0"],
  ["0","0","1","0","0"],
  ["0","0","0","1","1"]
]
Output: 3
```

#### Key Constraints and Edge Cases
- Consider edge cases where the grid may have the minimum size (1x1) or the maximum size (300x300).
- Check if the entire grid is water or land.
- Ensure the solution handles grids with irregular shapes of islands.

#### Possible Strategies and Algorithms
1. **Depth-First Search (DFS)**:
   - Iterate over each cell in the grid. When a land cell (`1`) is found, initiate a DFS to mark the entire island as visited.
   - Use a stack or recursion to explore all adjacent (horizontally and vertically) land cells.
   - Each DFS initiation counts as discovering a new island.

2. **Breadth-First Search (BFS)**:
   - Similar to DFS, but use a queue to explore all adjacent land cells level by level.
   - Start BFS from any unvisited land cell, marking all reachable cells as visited.

3. **Union-Find (Disjoint Set)**:
   - Use a union-find data structure to dynamically connect adjacent land cells.
   - Traverse the grid, uniting adjacent land cells, and count the number of root nodes to determine the number of islands.

#### Summary
The "Number of Islands" problem is a classic example of graph traversal techniques applied to a 2D grid. The main challenge is to correctly explore and mark all parts of an island while ensuring no cell is counted more than once. Efficient implementations typically use DFS or BFS, with Union-Find being an alternative for those familiar with the data structure.